In [1]:
import pandas
import os
import glob
import numpy
import tqdm

In [2]:
wells = glob.glob("/data1/santiago/BBBC021/experiments/original/Week*/All.csv")
meandf = pandas.read_csv("/data1/santiago/BBBC021/experiments/original/aggregate/Mean.csv")
stddf = pandas.read_csv("/data1/santiago/BBBC021/experiments/original/aggregate/STD.csv")
out_dir = "/data1/santiago/BBBC021/experiments/original/normalized/"

In [3]:
def get_plate(well):
    return os.path.basename(well[:-12])

In [4]:
def get_id(well):
    return os.path.basename(well[:-8])

In [5]:
for well in tqdm.tqdm(wells):
    if os.stat(well).st_size > 1:
        welldf = pandas.read_csv(well)
        mean = meandf[meandf["Plate"] == get_plate(well)].drop("Plate", axis=1)
        std = stddf[stddf["Plate"] == get_plate(well)].drop("Plate", axis=1)
        assert len(mean.columns) == len(std.columns) and len(mean) == 1 and len(std) == 1
        welldf = welldf.select(lambda c: c in mean.columns and c in std.columns, axis=1)
        meanarr = mean.values
        stdarr = std.values
        wellarr = welldf.values
        normalized = (wellarr - meanarr) / stdarr
        normaldf = welldf
        normaldf[:] = normalized
        normaldf.to_csv(os.path.join(out_dir, get_id(well) + ".csv"), index=False)

100%|██████████| 3300/3300 [1:14:05<00:00,  1.32s/it]
